**Импортируем все необходимые инструменты из библиотек**

In [101]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor as RF
from sklearn.metrics import r2_score as r2

**Импортируем выборку train из csv файла**

In [102]:
data = pd.read_csv('input/train.csv')

In [103]:
data.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


In [104]:
data.shape

(10000, 20)

**Корректируем данные в train**

In [105]:
data.loc[data['HouseYear'] == 20052011, 'HouseYear'] = 2008
data.loc[data['HouseYear'] == 4968, 'HouseYear'] = 1968

In [106]:
data.loc[data['Rooms'] > 5, :]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
377,5927,57,10.0,59.056975,36.223072,10.0,22,22.0,2002,0.090799,B,B,74,19083,2,NaN,5,15,B,317265.323792
1454,8491,1,19.0,42.006046,21.779288,7.0,17,17.0,2014,0.007122,B,B,1,264,0,NaN,0,1,B,78364.616704
2170,14003,99,6.0,59.414334,38.702244,6.0,7,9.0,1969,0.033494,B,B,66,10573,1,1322.0,3,8,B,229661.964416
8849,14865,9,10.0,60.871266,38.420681,10.0,3,2.0,1994,0.161532,B,B,25,5648,1,30.0,2,4,B,172329.270863


In [107]:
data.loc[data['Id'] == 5927, 'Rooms'] = 2
data.loc[data['Id'] == 8491, 'Rooms'] = 1
data.loc[data['Id'] == 14003, 'Rooms'] = 2
data.loc[data['Id'] == 14865, 'Rooms'] = 1

In [108]:
mean_square = data.groupby('Rooms', as_index=False)[['Square']].mean().rename(columns={'Square':'mean_square'})

In [109]:
data = pd.merge(data, mean_square, on='Rooms', how='left')

In [110]:
data.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,mean_square
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,...,B,33,7976,5,NaN,0,11,B,184966.930730,56.789475
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,...,B,46,10309,1,240.0,1,16,B,300009.450063,76.903234
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,...,B,34,7759,0,229.0,1,3,B,220925.908524,56.789475
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,...,B,23,5735,3,1084.0,0,5,B,175616.227217,56.789475
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,...,B,35,5776,1,2078.0,2,4,B,150226.531644,41.328735


In [111]:
data.loc[data['Square'] < 15, 'Square'] = data['mean_square']

**Делим датасет на 2: train и valid**

In [112]:
train, valid = train_test_split(data, test_size=0.3, random_state=42)

**Функция по добавлению количества квартир в округе и признака большого округа в датасет**

In [113]:
distr_info1 = train['DistrictId'].value_counts().reset_index().\
    rename(columns={'index':'DistrictId', 'DistrictId':'large_district'})

In [114]:
distr_info1['is_large'] = (distr_info1['large_district'] > 100).astype(int)

In [115]:
def add_distr_info(df, distr_info1):
    df = pd.merge(df, distr_info1, on='DistrictId', how='left')
    df['is_large'] = df['is_large'].fillna(0)
    df['large_district'] = df['large_district'].fillna(1)
    return df

**Добавляем данные по округу в train и в valid**

In [116]:
train = add_distr_info(train, distr_info1)

In [117]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7000 entries, 0 to 6999
Data columns (total 23 columns):
Id                7000 non-null int64
DistrictId        7000 non-null int64
Rooms             7000 non-null float64
Square            7000 non-null float64
LifeSquare        5514 non-null float64
KitchenSquare     7000 non-null float64
Floor             7000 non-null int64
HouseFloor        7000 non-null float64
HouseYear         7000 non-null int64
Ecology_1         7000 non-null float64
Ecology_2         7000 non-null object
Ecology_3         7000 non-null object
Social_1          7000 non-null int64
Social_2          7000 non-null int64
Social_3          7000 non-null int64
Healthcare_1      3642 non-null float64
Helthcare_2       7000 non-null int64
Shops_1           7000 non-null int64
Shops_2           7000 non-null object
Price             7000 non-null float64
mean_square       7000 non-null float64
large_district    7000 non-null int64
is_large          7000 non-null int3

In [118]:
valid = add_distr_info(valid, distr_info1)

In [119]:
valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 0 to 2999
Data columns (total 23 columns):
Id                3000 non-null int64
DistrictId        3000 non-null int64
Rooms             3000 non-null float64
Square            3000 non-null float64
LifeSquare        2373 non-null float64
KitchenSquare     3000 non-null float64
Floor             3000 non-null int64
HouseFloor        3000 non-null float64
HouseYear         3000 non-null int64
Ecology_1         3000 non-null float64
Ecology_2         3000 non-null object
Ecology_3         3000 non-null object
Social_1          3000 non-null int64
Social_2          3000 non-null int64
Social_3          3000 non-null int64
Healthcare_1      1560 non-null float64
Helthcare_2       3000 non-null int64
Shops_1           3000 non-null int64
Shops_2           3000 non-null object
Price             3000 non-null float64
mean_square       3000 non-null float64
large_district    3000 non-null float64
is_large          3000 non-null fl

**Обозначаем признаки, по которым будет работать модель**

In [120]:
feats = ['Rooms', 'Square', 'HouseYear', 'Ecology_1', 'Floor', 'large_district', 'Helthcare_2', 'Social_3' ]

**Строим модель и определяем параметры модели**

In [121]:
model = RF(n_estimators=28, max_depth=18, random_state=42)

**Обучаем модель**

In [122]:
model.fit(train.loc[:, feats], train['Price'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=18,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=28, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [123]:
pred_train = model.predict(train.loc[:, feats])

In [124]:
pred_valid = model.predict(valid.loc[:, feats])

In [125]:
r2(train['Price'], pred_train)

0.951858831079612

In [126]:
r2(valid['Price'], pred_valid)

0.7116803374330751

**Подбор признаков и параметров для построения модели**

Полученные результаты по r2 valid путем подбора признаков:

feats = ['Rooms', 'Square'] - 0.33

feats = ['Rooms', 'Square', 'Floor' ] - 0.37

feats = ['Rooms', 'Square', 'Floor', 'DistrictId' ] - 0.63

feats = ['Rooms', 'Square', 'Floor', 'DistrictId', 'Social_1' ] - 0.66

feats = ['Rooms', 'Square', 'Floor', 'DistrictId', 'HouseYear', 'large_district' ] - 0.67

feats = ['Rooms', 'Square', 'HouseYear', 'Ecology_1', 'Floor', 'large_district' ] - 0.69

feats = ['Rooms', 'Square', 'HouseYear', 'Ecology_1', 'Floor', 'large_district', 'Helthcare_2' ] - 0.70

feats = ['Rooms', 'Square', 'HouseYear', 'Ecology_1', 'Floor', 'large_district', 'Helthcare_2', 'Social_3' ] - 0.709

Потом были скорректированы параметры модели RF (n_estimators, max_depth) и r2 на valid стал равен 0.711

**Импортируем выборку test из csv файла**

In [127]:
test = pd.read_csv('input/test.csv')

In [128]:
test.shape

(5000, 19)

In [129]:
test.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,B,B,11,2748,1,NaN,0,0,B
1,15856,74,2.0,69.263183,NaN,1.0,6,1.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B
2,5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,B,B,30,7538,87,4702.0,5,5,B
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,B,B,23,4583,3,NaN,3,3,B
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,B,B,2,629,1,NaN,0,0,A


**Добавляем данные по округу в test**

In [130]:
test = add_distr_info(test, distr_info1)

In [131]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 21 columns):
Id                5000 non-null int64
DistrictId        5000 non-null int64
Rooms             5000 non-null float64
Square            5000 non-null float64
LifeSquare        3959 non-null float64
KitchenSquare     5000 non-null float64
Floor             5000 non-null int64
HouseFloor        5000 non-null float64
HouseYear         5000 non-null int64
Ecology_1         5000 non-null float64
Ecology_2         5000 non-null object
Ecology_3         5000 non-null object
Social_1          5000 non-null int64
Social_2          5000 non-null int64
Social_3          5000 non-null int64
Healthcare_1      2623 non-null float64
Helthcare_2       5000 non-null int64
Shops_1           5000 non-null int64
Shops_2           5000 non-null object
large_district    5000 non-null float64
is_large          5000 non-null float64
dtypes: float64(9), int64(9), object(3)
memory usage: 859.4+ KB


In [138]:
test['Price'] = model.predict(test.loc[:, feats])

In [140]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 22 columns):
Id                5000 non-null int64
DistrictId        5000 non-null int64
Rooms             5000 non-null float64
Square            5000 non-null float64
LifeSquare        3959 non-null float64
KitchenSquare     5000 non-null float64
Floor             5000 non-null int64
HouseFloor        5000 non-null float64
HouseYear         5000 non-null int64
Ecology_1         5000 non-null float64
Ecology_2         5000 non-null object
Ecology_3         5000 non-null object
Social_1          5000 non-null int64
Social_2          5000 non-null int64
Social_3          5000 non-null int64
Healthcare_1      2623 non-null float64
Helthcare_2       5000 non-null int64
Shops_1           5000 non-null int64
Shops_2           5000 non-null object
large_district    5000 non-null float64
is_large          5000 non-null float64
Price             5000 non-null float64
dtypes: float64(10), int64(9), obj

In [141]:
test.loc[:,['Id','Price']].to_csv('YRadkovskaya_predictions.csv', index = None)